# ASSIGNMENT 13

In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import update
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import select

In [2]:
# Read the following data set:
# https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
df = pd.read_csv(url)
df.columns=['age','workclass','fnlwgt','education','educationNum','marital_status','occupation','relationship','race','sex','capital_gain','capital_loss','hours_per_week','native_country','income']
df.head()

,age,workclass,fnlwgt,education,educationNum,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [3]:
# Removing all the blank spaces from the columns
def trimAllColumns(df):
    trimStrings = lambda x: x.strip() if type(x) is str else x
    return df.applymap(trimStrings)
df = trimAllColumns(df)
# df.head()

In [4]:
# Q1. Create an sqlalchemy engine using a sample from the data set

# Taking out a sample from the data
samp = df.sample(n=250)

# Creating an sqlalchemy engine
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=False)
samp.to_sql('adult_data', con=engine, if_exists='replace',index=True,chunksize = 62)

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String

# Creation of Adult Class
class Adult(Base):
    __tablename__ = 'adult_data'

    index = Column(Integer, primary_key=True)
      
    age = Column(Integer)
    workclass = Column(String)
    fnlwgt = Column(Integer)
    education = Column(String)
    educationNum = Column(Integer)
    marital_status = Column(String)
    occupation = Column(String)
    relationship = Column(String)
    race = Column(String)
    sex = Column(String)
    capital_gain = Column(Integer)
    capital_loss = Column(Integer)
    hours_per_week = Column(Integer)
    native_country = Column(String)
    income = Column(String)

    def __repr__(self):
        return "<Adult(age='{0}', workclass='{1}', fnlwgt='{2}',education='{3}',educationNum='{4}',marital_status='{5}',occupation='{6}',relationship='{7}',race='{8}',sex='{9}',capital_gain='{10}',capital_loss='{11}',hours_per_week='{12}',native_country,income='{13}')>".format(self.age,self.workclass,self.fnlwgt,self.education,self.educationNum,self.marital_status,self.occupation,self.relationship,self.race,self.sex,self.capital_gain,self.capital_loss,self.hours_per_week,self.native_country,self.income)

# Initiating a session
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

for adult in session.query(Adult):
     print(adult) 

<Adult(age='36', workclass='Federal-gov', fnlwgt='68781',education='Some-college',educationNum='10',marital_status='Married-civ-spouse',occupation='Adm-clerical',relationship='Husband',race='White',sex='Male',capital_gain='0',capital_loss='0',hours_per_week='35',native_country,income='United-States')>
<Adult(age='62', workclass='?', fnlwgt='199198',education='11th',educationNum='7',marital_status='Divorced',occupation='?',relationship='Not-in-family',race='Black',sex='Female',capital_gain='0',capital_loss='0',hours_per_week='40',native_country,income='United-States')>
<Adult(age='22', workclass='Private', fnlwgt='219086',education='Some-college',educationNum='10',marital_status='Married-civ-spouse',occupation='Sales',relationship='Husband',race='White',sex='Male',capital_gain='0',capital_loss='0',hours_per_week='55',native_country,income='United-States')>
<Adult(age='38', workclass='Local-gov', fnlwgt='287658',education='Masters',educationNum='14',marital_status='Divorced',occupation='

In [5]:
# Q2. Write two basic update queries
# First Update Query-Updating the marital status of all the persons having age 50 

# Printing the records of persons aging 50
for adult in session.query(Adult).filter_by(age=50):
     print(adult.age,adult.marital_status) 
        
# Updating the marital status as married
session.query(Adult).filter(Adult.age==50).update({'marital_status': 'Married'})

print("-"*80,"\n AFTER UPDATION\n","-"*80)

# Printing the updated records
for adult in session.query(Adult).filter_by(age=50):
     print(adult.age,adult.marital_status) 

50 Married-civ-spouse
50 Never-married
50 Married-civ-spouse
50 Married-civ-spouse
50 Married-civ-spouse
50 Married-civ-spouse
50 Separated
-------------------------------------------------------------------------------- 
 AFTER UPDATION
 --------------------------------------------------------------------------------
50 Married
50 Married
50 Married
50 Married
50 Married
50 Married
50 Married


In [6]:
# Second Update Query - Updating the race of persons hailing from Mexico
# Printing the records of people hailing from  Mexico
for adult in session.query(Adult).filter_by(native_country='Mexico'):
     print(adult.race,adult.native_country) 
        
# Updating the race as Black
session.query(Adult).filter(Adult.native_country=='Mexico').update({'race': 'Black'})

print("-"*80,"\n AFTER UPDATION\n","-"*80)

# Printing the updated records
for adult in session.query(Adult).filter_by(native_country='Mexico'):
     print(adult.race,adult.native_country) 

White Mexico
Other Mexico
White Mexico
White Mexico
White Mexico
-------------------------------------------------------------------------------- 
 AFTER UPDATION
 --------------------------------------------------------------------------------
Black Mexico
Black Mexico
Black Mexico
Black Mexico
Black Mexico


In [7]:
# Q3. Write two delete queries

# I- Deleting the records of people holding a doctorate
for adult in session.query(Adult).filter_by(education='Doctorate'):
     print(adult.age,adult.education,adult.workclass)
        
session.query(Adult).filter_by(education='Doctorate').delete()        

print("-"*80,"\n AFTER DELETION\n","-"*80)

# Trying to print After Deletion
for adult in session.query(Adult).filter_by(education='Doctorate'):
     print(adult.age,adult.education,adult.workclass)

58 Doctorate Private
31 Doctorate Local-gov
50 Doctorate Self-emp-not-inc
36 Doctorate Self-emp-not-inc
43 Doctorate Private
-------------------------------------------------------------------------------- 
 AFTER DELETION
 --------------------------------------------------------------------------------


In [8]:
# II - Deleting the records of persons having the age of 17
for adult in session.query(Adult).filter_by(age=17):
     print(adult.age,adult.sex,adult.native_country)
        
session.query(Adult).filter_by(age=17).delete()        

print("-"*80,"\n AFTER DELETION\n","-"*80)

# Trying to print After Deletion
for adult in session.query(Adult).filter_by(age=17):
     print(adult.age,adult.sex,adult.native_country)

17 Female United-States
-------------------------------------------------------------------------------- 
 AFTER DELETION
 --------------------------------------------------------------------------------


In [9]:
# Q4. Write two filter queries

# I. Filtering all the people woking in  the private sector
for adult in session.query(Adult).filter(Adult.workclass =='Private'):
    print(adult.age,adult.workclass,adult.sex,adult.education,adult.native_country)

22 Private Male Some-college United-States
18 Private Female HS-grad United-States
45 Private Male Bachelors United-States
55 Private Male HS-grad United-States
28 Private Male Bachelors United-States
23 Private Male Some-college United-States
55 Private Male Bachelors United-States
54 Private Male 11th United-States
47 Private Male HS-grad United-States
30 Private Male Some-college United-States
50 Private Male HS-grad United-States
43 Private Male HS-grad Germany
53 Private Female 12th United-States
46 Private Male Some-college United-States
26 Private Male Assoc-voc United-States
31 Private Male HS-grad United-States
20 Private Female Assoc-acdm United-States
44 Private Female HS-grad United-States
22 Private Male HS-grad United-States
56 Private Male HS-grad United-States
43 Private Male HS-grad United-States
42 Private Male Bachelors United-States
25 Private Male Some-college Outlying-US(Guam-USVI-etc)
24 Private Male 11th United-States
40 Private Female Preschool United-States
19

In [10]:
from sqlalchemy import and_
# II. Filtering the females working having masters degree
for adult in session.query(Adult).filter(and_(Adult.sex =='Female',Adult.education =='Masters')):
    print(adult.age,adult.sex,adult.education,adult.workclass,adult.race,adult.native_country,adult.hours_per_week)

48 Female Masters Local-gov White United-States 52
28 Female Masters State-gov White United-States 40
32 Female Masters Private White United-States 20
51 Female Masters Local-gov White United-States 40
47 Female Masters Local-gov White United-States 45


In [11]:
# Q5. Write two function queries
from sqlalchemy import func

# I- Function-Count of the people holding a bachelors degree 
print("Count of people having Bachelors Degree:")
for masters_count in session.query(func.count(Adult.index)).filter_by(education='Bachelors'):
    print(masters_count)

# II- Function-Avg hours_per_work of all the females employed in the Private sector    
for avg_age in session.query(func.avg(Adult.hours_per_week)).filter(and_(Adult.sex =='Female',Adult.workclass =='Private')):
    print("\nAverage work hours of all females employed in the private sector:" ,avg_age)     

Count of people having Bachelors Degree:
(37,)

Average work hours of all females employed in the private sector: (33.96875,)
